Welcome to NormalFramework:BACnet
===============================

We hope you enjoy your time here.  

Because NF is made up a set of gRPC services, you can interact with these services directly and not just through the UI.  We have prepared this set of notebooks to provide some examples of common tasks you can accomplish using Jupyter.

This notebook is the first of a few notebooks showing how you can interact with a BACnet network via gRPC to find and read devices.


Prerequisites
---------------

If you are trying to follow this guide, you should have NF:BACnet running somewhere it has a BACnet device to talk to.  A simulated device is fine for testing; you can use the `stevedh/bactools` docker image which has a prebuilt copy of bacnet-stack, which includes a server.  

Getting Started
------------------

This notebook can be run top to bottom; I've included lots of comments as well as markdown sections explaining each section.


Making the connection
=======

Since we're going to interact over gRPC, you need to import the compilied protobuf files.  Since you're in Jupyter, those are already installed.  If you want to run this from a command-line script, you'll have to get these from our Pypi page.

In [1]:
import os
import grpc
# bacnet_pb2 contains the message types, while bacnet_pb2_grpc has the RPC stubs
from normalgw.bacnet import bacnet_pb2, bacnet_pb2_grpc, bacenum_pb2

In [2]:
# the environment in docker-compose has the right address to connect to the BACnet service on.  
channel = grpc.insecure_channel('localhost:8080')
stub = bacnet_pb2_grpc.BacnetStub(channel)


Configuring BACnet Settings
========================

You might need to configure BACnet settings in order to communicate; for instance, changing the interface in use or the default port number; or register as a foreign device.  `NF:BACnet` exports a configuration API.

In [3]:
# create a stub for the configuration service.  
# It can use the same channel as the BACnet service.
config = bacnet_pb2_grpc.ConfigurationStub(channel)
# this will print out the full configuration.  it includes the
# local interface in use, BACnet port, other available interfaces,
# and settings for the local device object.
config.GetConfiguration(bacnet_pb2.GetConfigurationRequest())

# some of these can be updated using SetConfiguration: 
#   local_ifname, port, bbmd_address, bbmd_port, bbmd_ttl, 
#   device_instance, device_name

local_ifname: "enp1s0"
local_if_address: "192.168.103.185"
local_bcast_address: "192.168.103.255"
available_interfaces {
  name: "lo"
  addresses: "127.0.0.1/8"
}
available_interfaces {
  name: "enp1s0"
  addresses: "192.168.103.185/24"
}
available_interfaces {
  name: "tailscale0"
  addresses: "100.73.103.16/32"
}
available_interfaces {
  name: "lxdbr0"
  addresses: "10.119.241.1/24"
}
available_interfaces {
  name: "edibr0"
  addresses: "10.75.94.1/24"
}
available_interfaces {
  name: "br-3649c2ade53e"
  addresses: "172.20.0.1/16"
}
available_interfaces {
  name: "br-9c3da1b6ef22"
  addresses: "172.19.0.1/16"
}
available_interfaces {
  name: "docker0"
  addresses: "172.17.0.1/16"
}
available_interfaces {
  name: "br-640772e2395b"
  addresses: "172.18.0.1/16"
}
port: 47808
device_instance: 10
device_name: "NF"

Discovering Devices
================

The first thing you'll generally want to do is to find a device to talk to.  The BacnetStub service has methods for sending WhoIs, which requires a WhoIsRequest argument.  Generally, all gRPC methods take one request argument, and return one reply.  

To make things a little more complicated, the WhoIs method returns a stream response.  This is because it is a BACnet "unconfirmed" service, and so there is no way to tell when the request is "done".  In `NF:BACnet`, the default will return all I-Am messages for three seconds and then close the channel; we'll show you how to change this in a minute.

If you are running the bacnet-stack simulator, you should see one device with Device ID 260001 in response to your Who-Is.  The Jupyter cell should execute for three seconds and then return.

In [ ]:
for response in stub.WhoIs(bacnet_pb2.WhoIsRequest(low_limit=0xffffffff, high_limit=0xffffffff, options={'timeout':3})):
    print (response)

In [4]:
# you might want to target only a specific range of devices.  
# This can be useful for large networks, where you want to 
# block-scan only a portion of the ID space at a time to 
# prevent broadcast storms.
request = bacnet_pb2.WhoIsRequest(low_limit=260001, high_limit=260001, options={'block_lower_priority':True})
for response in stub.WhoIs(request):
    print (response)

device_address {
  mac: "\300\250g\222\272\300"
  max_apdu: 1476
  device_id: 260001
}



_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Received RST_STREAM with error code 0"
	debug_error_string = "{"created":"@1630526809.462763580","description":"Error received from peer ipv4:127.0.0.1:8080","file":"src/core/lib/surface/call.cc","file_line":1069,"grpc_message":"Received RST_STREAM with error code 0","grpc_status":13}"
>

In [ ]:
# many operations can take an OperationOptions to control the request processing.
# for instance, you may want a faster timeout if you are on a faster network.
request = bacnet_pb2.WhoIsRequest(options=bacnet_pb2.OperationOptions(timeout=.5))
for response in stub.WhoIs(request):
    print (response)

In [ ]:
# another useful skill is to send a unicast Who-Is.  You can use this 
# to implement range-scanning if NF:BACnet's broadcasts can't reach the 
# BACnet network.
#
# for this example to work, you probably need to edit this code to 
# contain the real IP of a BACnet device.
request = bacnet_pb2.WhoIsRequest(
    target=bacnet_pb2.DeviceAddress(
        # the MAC is the BACnet MAC -- a bytes array
        # since we are using BACnet/IP (Annex J) this is always 
        # a six-element array of IPv4 address + port number
        mac=bytes([192, 168, 103, 146, 0xBA, 0xC0]),
        
        # adr and net may be useful if you are trying to get a
        # router to forward your packet; but you'll need to fill 
        # these in
        adr=bytes([]),
        net=0,
        max_apdu=1496,
    )
)
for response in stub.WhoIs(request):
    print (response)

Reading From Devices (for Discovery)
==================

Once you have found a device (or before you have too) you probably want to use find out some details about the devices on your BACnetwork.  The BACnet service exports both ReadProperty and ReadPropMultiple services for reading from devices.

While generally, `NF:BACnet` exports BACnet semantics directly into gRPC, ReadPropMultiple has some differences which usually make it easier to write reliable software since vendors' support for ReadPropertyMultiple varies.  This includes:
1. If a response is too large, it will be broken into smaller requests.  Although NF:BACnet does not use segementation, this means that practically you can make as large a ReadPropMultiple request as you like.
1. If an Abort indicating ReadPropertyMultiple is not supported is received, NF:BACnet will retry using ReadProperty.  This can be quite slow.

In [ ]:
# We first need to get a DeviceAddress to talk to. 
# Usually these are saved from one of your WhoIs 
# requests, in this example I will use "dynamic binding" 
# to ask NF:BACnet to look it up during the request.
target_id = 260001
request = bacnet_pb2.ReadPropertyRequest(
    # by only filling in the device_id, we request dynamic 
    # binding -- the stack will do a Who-Is on your behalf 
    # and cache the result
    device_address=bacnet_pb2.DeviceAddress(device_id=target_id),
    # try to read the Device Name
    object_id=bacnet_pb2.ObjectId(object_type=bacenum_pb2.OBJECT_DEVICE, instance=target_id),
    property_id=bacenum_pb2.PROP_OBJECT_NAME,
    # have to send BACNET Array All since name is not an array.
    array_index=0xffffffff
)

# execute the request
# if this works for you, the response type is nearly the same as 
# for ReadPropMultiple to simplify processing
# note that the return type of Value (ApplicationDataValue) can be a little clunky;
# however it returns the BACnet type information
reply = stub.ReadProperty(request)
print (reply)
print ("Device name:", reply.value.character_string)

In [ ]:
# Read the object list in one request.
# 
# One downside of this approach is it will fail on 
# devices with large object lists since segemntation is not supported
# For those, use ReadPropMultiple with one read for each index -
# it works just as well.
target_id = 260001
request = bacnet_pb2.ReadPropertyRequest(
    # by only filling in the device_id, we request dynamic 
    # binding -- the stack will do a Who-Is on your behalf 
    # and cache the result
    device_address=bacnet_pb2.DeviceAddress(device_id=target_id),
    # try to read the Device Name
    object_id=bacnet_pb2.ObjectId(object_type=bacenum_pb2.OBJECT_DEVICE, instance=target_id),
    property_id=bacenum_pb2.PROP_OBJECT_LIST,
    # have to send BACNET Array All since name is not an array.
    array_index=0xffffffff
)
reply = stub.ReadProperty(request)

# the reply here is an array value, where each element is ObjectId type.
# since gRPC treats zero as the blank value, Analog Input 
# objects don't have their names pretty printed; but they are there.
for prop in reply.value.array:
    print (prop.object_id)

In [ ]:
# finally, you might want to check if a particular IP/port combo is running BACnet
# using the wildcard BACnet ID.
# N.B.: this can start to trigger IDS alerts if you use this too aggressively.
# https://nmap.org/nsedoc/scripts/bacnet-info.html

# edit these for your network.
target_ip = [192, 168, 103, 146]
target_port = 47808 # standard BACnet port

target_id = 4194303 # wildcard device id.  added in 135-2001a; most devices support.

# construct the request
request = bacnet_pb2.ReadPropertyRequest(
    # this time only fill in the "mac"
    device_address=bacnet_pb2.DeviceAddress(
        mac=bytes(target_ip + [(target_port >> 8) & 0xff, target_port & 0xff]),
        max_apdu=1496
    ),
    # try to read the Device object's Object Identifier
    object_id=bacnet_pb2.ObjectId(object_type=bacenum_pb2.OBJECT_DEVICE, instance=target_id),
    property_id=bacenum_pb2.PROP_OBJECT_IDENTIFIER,
    # have to send BACNET Array All since name is not an array.
    array_index=0xffffffff
)
reply = stub.ReadProperty(request)
print (reply)

# if this works, the object_id instance will be the device_id 
# of the BACnet device at this address
# you can use this to make further Read requests on the Name, Object list, and so on.
print (reply.value.object_id.instance)